### 参考文献
https://qiita.com/ceptree/items/9a473b5163d5655420e8 <br/>
https://yul.hatenablog.com/entry/2019/01/07/152738 <br/>
数学もきっちりかいてある https://www.hellocybernetics.tech/entry/2016/04/21/132645 <br/>
北大の資料情報理論の基礎http://www-ikn.ist.hokudai.ac.jp/~kida/lecture/IT_3.pdf <br/>
pythonでクロス集計を作成https://datumstudio.jp/blog/pythonでクロス集計を実行 <br/>
DataFrame から ピボットテーブルを作成－Python pandas http://ailaby.com/dataframe_pivot/#id2 <br/>
KL情報量のnumpyを用いた計算コードhttp://sucrose.hatenablog.com/entry/2013/11/10/001555 <br/>
Pythonで浮動小数点数floatの誤差を考慮して比較（math.isclose）https://note.nkmk.me/python-math-isclose/

# KL divergence練習

- 確率が0になると計算ができなかったので、0が一つでもある行列に対しては、すべてのカウントを+1しました。

In [8]:
import pandas as pd
df = pd.read_csv('sample.csv', index_col=0)
df

,TF1,TF2,TF3,TF4,expression
#gene,,,,,
A,1,1,1,0,1
B,1,1,1,0,1
C,1,0,0,1,0
D,0,0,0,0,0
E,1,1,1,0,1
F,1,0,0,0,0
G,1,1,1,1,1
H,0,0,0,0,1
I,0,1,0,1,0


### TF1 & TF2について

In [9]:
T12 = df.drop(['TF3', 'TF4', 'expression'], axis=1)
#print(T12)
T12_l = T12.values.tolist()
#T12_l

In [10]:
sr = pd.Series(T12_l)
sr.value_counts() # それぞれの値の出現回数をカウントしてくれる。

[0, 0]    7
[1, 1]    5
[1, 0]    2
[0, 1]    1
dtype: int64

In [11]:
T12e = df.drop(['TF3', 'TF4'], axis=1)
print(T12)
T12e_l = T12e.values.tolist()
#T12e_l

       TF1  TF2
#gene          
A        1    1
B        1    1
C        1    0
D        0    0
E        1    1
F        1    0
G        1    1
H        0    0
I        0    1
J        0    0
K        0    0
L        0    0
M        0    0
N        1    1
O        0    0


In [12]:
sr = pd.Series(T12e_l)
sr.value_counts() # それぞれの値の出現回数をカウントしてくれる。

[0, 0, 0]    5
[1, 1, 1]    5
[1, 0, 0]    2
[0, 0, 1]    2
[0, 1, 0]    1
dtype: int64

DataFrameはざっくり言うと二次元配列に相当、一次元配列はSeries

二重確率行列を作りたいhttps://ja.wikipedia.org/wiki/二重確率行列

In [13]:
#df12=pd.DataFrame([[7,1, 8], [2, 5, 7], [9, 6, 15]], index=['TF1=0', 'TF1=1', 'each_sum_TF2'], columns=['TF2=0','TF2=1', 'each_sum_TF1'])

In [14]:
df12_ex0=pd.DataFrame([[(5+1)/12,(1+1)/12, (6+2)/12], [(2+1)/12, (0+1)/12, (2+2)/12], [(7+2)/12, (1+2)/12, (8+4)/12]], 
                      index=['TF1=0', 'TF1=1', 'each_sum_TF2'], columns=['TF2=0','TF2=1', 'each_sum_TF1'])
df12_ex0

,TF2=0,TF2=1,each_sum_TF1
TF1=0,0.50,0.166667,0.666667
TF1=1,0.25,0.083333,0.333333
each_sum_TF2,0.75,0.250000,1.000000


In [15]:
df12_ex1=pd.DataFrame([[(2+1)/11,(0+1)/11, (2+2)/11], [(0+1)/11, (5+1)/11, (5+2)/11], [(2+2)/11, (5+2)/11, (7+4)/11]], 
                      index=['TF1=0', 'TF1=1', 'each_sum_TF2'], columns=['TF2=0','TF2=1', 'each_sum_TF1'])
df12_ex1

,TF2=0,TF2=1,each_sum_TF1
TF1=0,0.272727,0.090909,0.363636
TF1=1,0.090909,0.545455,0.636364
each_sum_TF2,0.363636,0.636364,1.000000


In [16]:
df12_ex0.iloc[2, 1] #each_sum_TF2 and TF2=1

0.25

In [17]:
def shinkhorn(matrix0, matrix1):
    repeat = True
    while repeat:
        for i in range(0, 2):
            if matrix0.iloc[2, i] != matrix1.iloc[2, i]:
                for j in range(0, 2):
                    matrix0.iloc[j, i] = matrix0.iloc[j, i] / matrix0.iloc[2, i] * matrix1.iloc[2, i]
        matrix0.iloc[2, 0] = matrix0.iloc[0, 0] + matrix0.iloc[1, 0]
        matrix0.iloc[2, 1] = matrix0.iloc[0, 1] + matrix0.iloc[1, 1]
        matrix0.iloc[0, 2] = matrix0.iloc[0, 0] + matrix0.iloc[0, 1]
        matrix0.iloc[1, 2] = matrix0.iloc[1, 0] + matrix0.iloc[1, 1]
        
        for k in range(0, 2):
                if matrix0.iloc[k, 2] != matrix1.iloc[k, 2]:
                    for l in range(0, 2):
                        matrix0.iloc[l, k] = matrix0.iloc[l, k] / matrix0.iloc[l, 2] * matrix1.iloc[l, 2]
        matrix0.iloc[2, 0] = matrix0.iloc[0, 0] + matrix0.iloc[1, 0]
        matrix0.iloc[2, 1] = matrix0.iloc[0, 1] + matrix0.iloc[1, 1]
        matrix0.iloc[0, 2] = matrix0.iloc[0, 0] + matrix0.iloc[0, 1]
        matrix0.iloc[1, 2] = matrix0.iloc[1, 0] + matrix0.iloc[1, 1]
        
        repeat = False
        for i in range(0, 2):
            if (matrix0.iloc[2, i] - matrix1.iloc[2, i])**2 >= 0.0001:
                repeat = True          
        for k in range(0, 2):
            if (matrix0.iloc[k, 2] - matrix1.iloc[k, 2])**2 >= 0.0001:
                repeat = True
                
    matrix0.iloc[2, 2] = matrix0.iloc[2, 0] + matrix0.iloc[2, 1]  
    return matrix0

In [18]:
df12_ex0 #p12

,TF2=0,TF2=1,each_sum_TF1
TF1=0,0.50,0.166667,0.666667
TF1=1,0.25,0.083333,0.333333
each_sum_TF2,0.75,0.250000,1.000000


In [19]:
shinkhorn(df12_ex0.copy(), df12_ex1) #pp12

,TF2=0,TF2=1,each_sum_TF1
TF1=0,0.132231,0.231405,0.363636
TF1=1,0.231405,0.404959,0.636364
each_sum_TF2,0.363636,0.636364,1.000000


In [20]:
df12_ex1 #q12

,TF2=0,TF2=1,each_sum_TF1
TF1=0,0.272727,0.090909,0.363636
TF1=1,0.090909,0.545455,0.636364
each_sum_TF2,0.363636,0.636364,1.000000


In [21]:
import numpy as np

def kl(p, q):
    return np.sum(np.where(p != 0, p * np.log2(p / q), 0))

In [22]:
p12 = np.array([[(5+1)/12,(1+1)/12, (6+2)/12], [(2+1)/12, (0+1)/12, (2+2)/12], [(7+2)/12, (1+2)/12, (8+4)/12]])
pp12 = np.array([[0.132231, 0.231405, 0.363636], [0.2311405, 0.404959, 0.636364], [0.363636, 0.636364,  1.000000]])
q12 = np.array([[(2+1)/11,(0+1)/11, (2+2)/11], [(0+1)/11, (5+1)/11, (5+2)/11], [(2+2)/11, (5+2)/11, (7+4)/11]])

In [23]:
kl(q12,  p12)

1.7814420918511125

In [24]:
kl(q12, pp12) #interaction

0.27428156560774863

In [25]:
kl(pp12, p12) # 

1.5069622287613549

In [79]:
import math
print(math.isclose(kl(q12,  p12), kl(q12,  pp12) + kl(pp12,  p12),  abs_tol=0.01))

True


### TF1 & TF3について

In [27]:
T13 = df.drop(['TF2', 'TF4', 'expression'], axis=1)
#print(T13)
T13_l = T13.values.tolist()
#T13_l

T13e = df.drop(['TF2', 'TF4'], axis=1)
#print(T13)
T13e_l = T13e.values.tolist()
#T13e_l

In [28]:
sr = pd.Series(T13e_l)
sr.value_counts() # それぞれの値の出現回数をカウントしてくれる。

[0, 0, 0]    5
[1, 1, 1]    5
[1, 0, 0]    2
[0, 0, 1]    2
[0, 1, 0]    1
dtype: int64

In [29]:
df13_ex0=pd.DataFrame([[(5+1)/12, (2+1)/12, (7+2)/12], [(1+1)/12, (0+1)/12, (1+2)/12], [(6+2)/12, (2+2)/12, (8+4)/12]], 
                      index=['TF1=0', 'TF1=1', 'each_sum_TF3'], columns=['TF3=0','TF3=1', 'each_sum_TF1'])
df13_ex0 #p13

,TF3=0,TF3=1,each_sum_TF1
TF1=0,0.500000,0.250000,0.75
TF1=1,0.166667,0.083333,0.25
each_sum_TF3,0.666667,0.333333,1.00


In [30]:
df13_ex1=pd.DataFrame([[(2+1)/11, (0+1)/11 ,(2+2)/11], [(0+1)/11, (5+1)/11, (5+2)/11], [(2+2)/11, (5+2)/11, (7+4)/11]], 
                      index=['TF1=0', 'TF1=1', 'each_sum_TF3'], columns=['TF3=0','TF3=1', 'each_sum_TF1'])
df13_ex1 #q13

,TF3=0,TF3=1,each_sum_TF1
TF1=0,0.272727,0.090909,0.363636
TF1=1,0.090909,0.545455,0.636364
each_sum_TF3,0.363636,0.636364,1.000000


In [31]:
shinkhorn(df13_ex0.copy(), df13_ex1) #pp13

,TF3=0,TF3=1,each_sum_TF1
TF1=0,0.132231,0.231405,0.363636
TF1=1,0.231405,0.404959,0.636364
each_sum_TF3,0.363636,0.636364,1.000000


In [32]:
p13 = np.array([[(5+1)/12, (2+1)/12, (7+2)/12], [(1+1)/12, (0+1)/12, (1+2)/12], [(6+2)/12, (2+2)/12, (8+4)/12]])
pp13 = np.array([[0.132231, 0.231405, 0.363636], [0.2311405, 0.404959, 0.636364], [0.363636, 0.636364,  1.000000]])
q13 = np.array([[(2+1)/11,(0+1)/11, (2+2)/11], [(0+1)/11, (5+1)/11, (5+2)/11], [(2+2)/11, (5+2)/11, (7+4)/11]])

In [33]:
kl(q13,  p13)

1.7814420918511125

In [34]:
kl(q13, pp13)

0.27428156560774863

In [35]:
kl(pp13, p13)

1.506807506179914

In [80]:
print(math.isclose(kl(q13,  p13), kl(q13,  pp13) + kl(pp13,  p13),  abs_tol=0.01))

True


### TF1 & TF4 について

In [37]:
T14 = df.drop(['TF2', 'TF3', 'expression'], axis=1)
#print(T14)
T14_l = T14.values.tolist()
#T14_l

T14e = df.drop(['TF2', 'TF3'], axis=1)
#print(T14)
T14e_l = T14e.values.tolist()
#T14e_l

In [38]:
sr = pd.Series(T14e_l)
sr.value_counts() # それぞれの値の出現回数をカウントしてくれる。

[1, 0, 1]    4
[0, 1, 0]    3
[0, 0, 0]    3
[0, 0, 1]    1
[1, 1, 0]    1
[1, 1, 1]    1
[1, 0, 0]    1
[0, 1, 1]    1
dtype: int64

In [39]:
df14_ex0=pd.DataFrame([[3/8, 3/8, 6/8], [1/8, 1/8, 2/8], [4/8, 4/8, 8/8]], 
                      index=['TF1=0', 'TF1=1', 'each_sum_TF4'], columns=['TF4=0','TF4=1', 'each_sum_TF1'])
df14_ex0 #p14

,TF4=0,TF4=1,each_sum_TF1
TF1=0,0.375,0.375,0.75
TF1=1,0.125,0.125,0.25
each_sum_TF4,0.500,0.500,1.00


In [40]:
df14_ex1=pd.DataFrame([[1/7, 1/7, 2/7], [4/7, 1/7, 5/7], [5/7, 2/7, 7/7]], 
                      index=['TF1=0', 'TF1=1', 'each_sum_TF4'], columns=['TF4=0','TF4=1', 'each_sum_TF1'])
df14_ex1 #q14

,TF4=0,TF4=1,each_sum_TF1
TF1=0,0.142857,0.142857,0.285714
TF1=1,0.571429,0.142857,0.714286
each_sum_TF4,0.714286,0.285714,1.000000


In [41]:
shinkhorn(df14_ex0.copy(), df14_ex1) #pp14

,TF4=0,TF4=1,each_sum_TF1
TF1=0,0.204082,0.081633,0.285714
TF1=1,0.510204,0.204082,0.714286
each_sum_TF4,0.714286,0.285714,1.000000


In [42]:
p14 = np.array([[3/8, 3/8, 6/8], [1/8, 1/8, 2/8], [4/8, 4/8, 8/8]])
pp14 = np.array([[0.204082, 0.081633, 0.285714], [0.510204, 0.204082, 0.714286], [0.714286, 0.285714,  1.000000]])
q14 = np.array([[1/7, 1/7, 2/7], [4/7, 1/7, 5/7], [5/7, 2/7, 7/7]])

In [43]:
kl(q14,  p14)

1.7035682261113303

In [44]:
kl(q14, pp14)

0.06174187196002533

In [45]:
kl(pp14, p14)

1.6418265276716526

In [81]:
print(math.isclose(kl(q14,  p14), kl(q14,  pp14) + kl(pp14,  p14),  abs_tol=0.01))

True


### TF2 & TF3 について

In [47]:
T23 = df.drop(['TF2', 'TF4', 'expression'], axis=1)
#print(T23)
T23_l = T23.values.tolist()
#T23_l

T23e = df.drop(['TF2', 'TF4'], axis=1)
#print(T23)
T23e_l = T23e.values.tolist()
#T23e_l

In [48]:
sr = pd.Series(T23e_l)
sr.value_counts() # それぞれの値の出現回数をカウントしてくれる。

[0, 0, 0]    5
[1, 1, 1]    5
[1, 0, 0]    2
[0, 0, 1]    2
[0, 1, 0]    1
dtype: int64

In [49]:
df23_ex0=pd.DataFrame([[(5+1)/12, (1+1)/12, (6+2)/12], [(2+1)/12, (0+1)/12, (2+2)/12], [(7+2)/12, (1+2)/12, (8+4)/12]], 
                      index=['TF2=0', 'TF2=1', 'each_sum_TF3'], columns=['TF3=0','TF3=1', 'each_sum_TF2'])
df23_ex0 #p23

,TF3=0,TF3=1,each_sum_TF2
TF2=0,0.50,0.166667,0.666667
TF2=1,0.25,0.083333,0.333333
each_sum_TF3,0.75,0.250000,1.000000


In [50]:
df23_ex1=pd.DataFrame([[(2+1)/11, (0+1)/11, (2+2)/11], [(0+1)/11, (5+1)/11, (5+2)/11], [(2+2)/11, (5+2)/11, (7+4)/11]], 
                      index=['TF1=0', 'TF1=1', 'each_sum_TF4'], columns=['TF4=0','TF4=1', 'each_sum_TF1'])
df23_ex1 #q23

,TF4=0,TF4=1,each_sum_TF1
TF1=0,0.272727,0.090909,0.363636
TF1=1,0.090909,0.545455,0.636364
each_sum_TF4,0.363636,0.636364,1.000000


In [51]:
shinkhorn(df23_ex0.copy(), df23_ex1) #pp23

,TF3=0,TF3=1,each_sum_TF2
TF2=0,0.132231,0.231405,0.363636
TF2=1,0.231405,0.404959,0.636364
each_sum_TF3,0.363636,0.636364,1.000000


In [52]:
p23 = np.array([[(5+1)/12, (1+1)/12, (6+2)/12], [(2+1)/12, (0+1)/12, (2+2)/12], [(7+2)/12, (1+2)/12, (8+4)/12]])
pp23 = np.array([[0.132231, 0.231405, 0.363636], [0.231405, 0.404959, 0.636364], [0.363636, 0.636364,  1.000000]])
q23 = np.array([[(2+1)/11, (0+1)/11, (2+2)/11], [(0+1)/11, (5+1)/11, (5+2)/11], [(2+2)/11, (5+2)/11, (7+4)/11]])

In [53]:
kl(q23,  p23)

1.7814420918511125

In [54]:
kl(q23,  pp23)

0.2741315684212672

In [55]:
kl(pp23,  p23)

1.507314109551623

In [82]:
print(math.isclose(kl(q23,  p23), kl(q23,  pp23) + kl(pp23,  p23),  abs_tol=0.01))

True


### TF2 & TF4 について

In [57]:
T24 = df.drop(['TF1', 'TF3', 'expression'], axis=1)
#print(T24)
T24_l = T24.values.tolist()
#T24_l

T24e = df.drop(['TF1', 'TF3'], axis=1)
#print(T24)
T24e_l = T24e.values.tolist()
#T24e_l

In [58]:
sr = pd.Series(T24e_l)
sr.value_counts() # それぞれの値の出現回数をカウントしてくれる。

[1, 0, 1]    4
[0, 0, 0]    4
[0, 1, 0]    3
[1, 1, 0]    1
[0, 1, 1]    1
[0, 0, 1]    1
[1, 1, 1]    1
dtype: int64

In [59]:
df24_ex0=pd.DataFrame([[(4+1)/12, (3+1)/12, (7+2)/12], [(0+1)/12 ,(1+1)/12, (1+2)/12], [(4+2)/12, (4+2)/12, (8+4)/12]], 
                      index=['TF2=0', 'TF2=1', 'each_sum_TF4'], columns=['TF4=0','TF4=1', 'each_sum_TF2'])
df24_ex0 #p24

,TF4=0,TF4=1,each_sum_TF2
TF2=0,0.416667,0.333333,0.75
TF2=1,0.083333,0.166667,0.25
each_sum_TF4,0.500000,0.500000,1.00


In [60]:
df24_ex1=pd.DataFrame([[1/7, 1/7 ,2/7], [4/7, 1/7, 5/7], [5/7, 2/7, 7/7]], 
                      index=['TF2=0', 'TF2=1', 'each_sum_TF4'], columns=['TF4=0','TF4=1', 'each_sum_TF2'])
df24_ex1 #q24

,TF4=0,TF4=1,each_sum_TF2
TF2=0,0.142857,0.142857,0.285714
TF2=1,0.571429,0.142857,0.714286
each_sum_TF4,0.714286,0.285714,1.000000


In [61]:
shinkhorn(df24_ex0.copy(), df24_ex1) #pp24

,TF4=0,TF4=1,each_sum_TF2
TF2=0,0.237504,0.048210,0.285714
TF2=1,0.473833,0.240453,0.714286
each_sum_TF4,0.711338,0.288662,1.000000


In [62]:
p24 = np.array([[(4+1)/12, (3+1)/12, (7+2)/12], [(0+1)/12, (1+1)/12, (1+2)/12], [(4+2)/12, (4+2)/12, (8+4)/12]])
pp24 = np.array([[0.237504, 0.048210, 0.285714], [0.473833, 0.240453, 0.714286], [0.711338, 0.288662,  1.000000]])
q24 = np.array([[1/7, 1/7, 2/7], [4/7, 1/7, 5/7], [5/7, 2/7, 7/7]])

In [63]:
kl(q24,  p24)

1.9811017134831936

In [64]:
kl(q24,  pp24)

0.16622864737702492

In [65]:
kl(pp24,  p24)

1.8052046046154082

In [83]:
print(math.isclose(kl(q24,  p24), kl(q24,  pp24) + kl(pp24,  p24),  abs_tol=0.01))

True


### TF3 & TF4 について

In [67]:
T34 = df.drop(['TF1', 'TF2', 'expression'], axis=1)
#print(T34)
T34_l = T34.values.tolist()
#T34_l

T34e = df.drop(['TF1', 'TF2'], axis=1)
#print(T34)
T34e_l = T34e.values.tolist()
#T34e_l

In [68]:
sr = pd.Series(T34e_l)
sr.value_counts() # それぞれの値の出現回数をカウントしてくれる。

[0, 1, 0]    4
[1, 0, 1]    4
[0, 0, 0]    3
[1, 0, 0]    1
[0, 1, 1]    1
[0, 0, 1]    1
[1, 1, 1]    1
dtype: int64

In [69]:
df34_ex0=pd.DataFrame([[(3+1)/12, (4+1)/12, (7+2)/12], [(1+1)/12, (0+1)/12 ,(1+2)/12], [(4+2)/12 ,(4+2)/12, (8+4)/12]], 
                      index=['TF3=0', 'TF3=1', 'each_sum_TF4'], columns=['TF4=0','TF4=1', 'each_sum_TF3'])
df34_ex0 #p34

,TF4=0,TF4=1,each_sum_TF3
TF3=0,0.333333,0.416667,0.75
TF3=1,0.166667,0.083333,0.25
each_sum_TF4,0.500000,0.500000,1.00


In [70]:
df34_ex1=pd.DataFrame([[1/7, 1/7, 2/7], [4/7, 1/7, 5/7], [5/7, 2/7, 7/7]], 
                      index=['TF3=0', 'TF3=1', 'each_sum_TF4'], columns=['TF4=0','TF4=1', 'each_sum_TF3'])
df34_ex1 #q34

,TF4=0,TF4=1,each_sum_TF3
TF3=0,0.142857,0.142857,0.285714
TF3=1,0.571429,0.142857,0.714286
each_sum_TF4,0.714286,0.285714,1.000000


In [71]:
shinkhorn(df34_ex0.copy(), df34_ex1) #pp34

,TF4=0,TF4=1,each_sum_TF3
TF3=0,0.164835,0.120879,0.285714
TF3=1,0.552283,0.162003,0.714286
each_sum_TF4,0.717118,0.282882,1.000000


In [72]:
p34 = np.array([[(3+1)/12, (4+1)/12, (7+2)/12], [(1+1)/12, (0+1)/12, (1+2)/12], [(4+2)/12, (4+2)/12, (8+4)/12]])
pp34 = np.array([[0.164835, 0.120879, 0.285714], [0.552283, 0.162003, 0.714286], [0.717118, 0.282882,  1.000000]])
q34 = np.array([[1/7, 1/7, 2/7], [4/7, 1/7, 5/7], [5/7, 2/7, 7/7]])

In [73]:
kl(q34,  p34)

1.5525302849117648

In [74]:
kl(q34,  pp34)

0.007139119237834075

In [75]:
kl(pp34,  p34)

1.5513728582414896

In [84]:
print(math.isclose(kl(q34,  p34), kl(q34,  pp34) + kl(pp34,  p34),  abs_tol=0.01))

True
